In [1]:
from openpyxl import Workbook
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import os
import sys
import copy
import json
import sys


In [2]:
#Outputs all of the Stages, Counts, & Countries (Ex. Engage->Qualify)
if __name__ == "__main__":

    output_df = pd.DataFrame()

   
    won = 0
    directory = os.getcwd() + "/data"
    files = []
    for i in sorted(os.listdir(directory)):
        if os.path.isfile(os.path.join(directory,i)) and ('Product_Pipe' in i) and ('~' not in i):
            files.append(i)
    print(files)

 
#     if put in two excel sheets in commandline
#     if(len(sys.argv)>1):
#         #files = [sys.argv[1], sys.argv[2], sys.argv[3]]
#         files = sys.argv[1:]
#     else:
#         files = sorted(os.listdir(directory))
    

#     print(files)

    

    #first sheet we do value counts
    #prev = first sheet dataframe
    filename = files[0]
    

    f = os.path.join(directory, filename)
    print(f)
    df_prev = pd.read_excel(f, sheet_name='Sheet1')
    df_prev = pd.DataFrame(df_prev, columns=['Opp No', 'Sales Stage Name', 'Country', 'Opportunity Source', 'LIO Channel', 'Sector with GBE'])
    df_prev = df_prev.drop_duplicates(subset=["Opp No"], keep="last")
   

    filename = filename.split(".")[0][-5:]
 

    for i in range(1, len(files)):
        won = 0
        filename = files[i]
        f = os.path.join(directory, filename)
        df_current = pd.read_excel(f, sheet_name='Sheet1')
       
        df_current = df_current.drop_duplicates(subset=["Opp No"], keep="last")
        df_current = pd.DataFrame(df_current, columns=['Opp No', 'Sales Stage Name', 'Country', 'Opportunity Source', 'LIO Channel', 'Sector with GBE'])
        temp = df_current["Sales Stage Name"].value_counts()
        
        df_current = df_current.rename(columns= {'Opp No': "Opp No Cur", 'Sales Stage Name': "Sales Stage Name Cur"})
        df_compare = pd.merge(df_prev, df_current, how = "outer", left_on=df_prev["Opp No"], right_on = df_current["Opp No Cur"])

        df_compare = df_compare.fillna(0)
        df_compare.to_excel("test.xlsx")
        df_temp = pd.read_excel('funnel.xlsx')
        temp2 = pd.DataFrame(df_temp)
        temp2.to_excel('temp2.xlsx')
        L = []
    
 
        changes = {}
      
    
        newOpp_list = pd.DataFrame(columns=['Sales Stage Name'])

        #outputName = "compare" + str(i) + ".xlsx"
        #df_compare.to_excel(outputName)


        for index, row in df_compare.iterrows():

      
            Sales_Stage_Name_prev = row["Sales Stage Name"]
            Sales_Stage_Name_cur = row["Sales Stage Name Cur"]
  
            
            if(Sales_Stage_Name_prev != "Loss" or Sales_Stage_Name_cur != "Loss"):
                key = Sales_Stage_Name_cur
                file = filename.split(".")[0][-5:]
                dat= [{'Date': file, 'Changes': key, 'Count': '1'}]
                dd = pd.DataFrame(dat, columns = ['Date', 'Changes', 'Count'])
                L.append(dd)
            
                    
                    
                    
                
       #outside of loop
        df = pd.concat(L, ignore_index=True)
        df_temp = pd.read_excel('funnel.xlsx', index_col=[0])
        df_temp = pd.DataFrame(df_temp)
        frames = [df_temp, df]
        df_output = pd.concat(frames, ignore_index=True)
        df_merge = pd.read_excel('funnel_merge.xlsx', index_col=[0])
        df_merge = pd.DataFrame(df_merge)
        df_output = pd.merge(df_merge, df_output, how="outer", on=["Date", "Changes"])
        df_output.to_excel('funnel.xlsx')
        df_current = df_current.rename(columns= {'Opp No Cur': "Opp No", 'Sales Stage Name Cur': "Sales Stage Name"})
        df_prev = df_current
        
    
 
        
               
   

['Product_Pipe_09-27.xlsx', 'Product_Pipe_10-04.xlsx', 'Product_Pipe_10-11.xlsx', 'Product_Pipe_10-18.xlsx', 'Product_Pipe_10-25.xlsx', 'Product_Pipe_11-01.xlsx']
/Users/Jessica.Journeycake@ibm.com/Desktop/R2M_UI/data/Product_Pipe_09-27.xlsx


In [3]:
df2 = pd.read_excel('funnel.xlsx', index_col=False)
df2 = pd.DataFrame(df2)
df2 = df2.groupby(['Date', 'Changes'], as_index=False)["Count"].count()

df2 = df2[df2['Date'] != '08-10']
df2 = df2[df2['Date'] != '08-17']
df2 = df2[df2['Date'] != '08-24']
df2 = df2[df2['Date'] != '08-30']

df_sum = df2.drop(columns=['Changes'], axis=0)
df_sum = df2.groupby(['Date'], as_index=False)["Count"].sum()
df_sum = df_sum.rename(columns={'Count': "Total"})

df2 = pd.merge(df2, df_sum, on=["Date"])

df2["Average"] = (df2['Count'] / df2['Total']) * 100
print(df2.head(5))
df2.to_excel('funnel_output.xlsx')



    Date    Changes  Count  Total    Average
0  10-04    Closing      0    318   0.000000
1  10-04     Design     34    318  10.691824
2  10-04     Engage    156    318  49.056604
3  10-04       Loss      0    318   0.000000
4  10-04  Negotiate      0    318   0.000000
